# Learning From the Data, Homework 7

## Validation

In the following problems, use the data provided in the files `in.dta` and `out.dta` for Homework # 6. We are going to apply linear regression with a nonlinear transformation for classification (without regularization). The nonlinear transformation is given by $\phi_0$ through $\phi_7$ which transform $(x_1, x_2)$ into

$1 \qquad x_1 \qquad x_2 \qquad x_1^2 \qquad x_2^2 \qquad x_1x_2 \qquad |x_1 - x_2| \qquad |x_1 + x_2|$

To illustrate how taking out points for validation affects the performance, we will consider the hypotheses trained on $\mathcal{D}_\text{train}$ (without restoring the full $\mathcal{D}$ for training after validation is done).

### Problem 1

Split `in.dta` into training (first 25 examples) and validation (last 10 examples). Train on the 25 examples only, using the validation set of the 10 examples to select between five models that apply linear regression $\phi_0$ through $\phi_k$, with $k = 3, 4, 5, 6, 7$. For which model is the classification error on the validation set smallest?

In [2]:
import numpy as np
import pandas as pd

In [13]:
def transform_data(X1, X2, k=7):
    Z = np.column_stack((np.ones(X1.shape), 
                     X1, 
                     X2, 
                     X1**2, 
                     X2**2,
                     X1*X2,
                     np.abs(X1 - X2),
                     np.abs(X1 + X2)))
    return Z[:, 0:k+1]

def linear_regression(data, W=None, lamd=0, k=7):
    # Extract and transform data
    X1 = data[:,0]
    X2 = data[:,1]
    Y = data[:,[2]]
    
    Z = transform_data(X1, X2, k=k)

    # Compute W if not given
    if W is None:
        Z_square = np.dot(Z.T, Z)
        W = np.dot(np.linalg.inv(Z_square + np.identity(Z_square.shape[0])*lamd), np.dot(Z.T, Y))

    
    Y_hat = np.sign(np.dot(W.T, Z.T))
    error = np.sum(Y_hat.T != Y)/Y.shape[0]
    return error, W

def run_regression(train, test, lamd=0, k=7):
    # Compute E_in and E_out
    E_in, W_lin = linear_regression(train, lamd=lamd, k=k)
    E_out, _ = linear_regression(test, W=W_lin, k=k)
    return E_in, E_out

def run_experiment(train, test):
    for k in range(3,8):
        print(k, run_regression(train, test, k=k))

# Read in data
train = pd.read_table("http://work.caltech.edu/data/in.dta", delim_whitespace=True, header=None).values
test = pd.read_table("http://work.caltech.edu/data/out.dta", delim_whitespace=True, header=None).values

D_train = train[0:25, :]
D_val = train[25:, :]
D_test = test

run_experiment(D_train, D_val)

3 (0.44, 0.29999999999999999)
4 (0.32000000000000001, 0.5)
5 (0.080000000000000002, 0.20000000000000001)
6 (0.040000000000000001, 0.0)
7 (0.040000000000000001, 0.10000000000000001)


Answer: [**d**] $k=6$

### Problem 2

Evaluate the out-of-sample classification error using `out.dta` on the 5 models to see how well the validation set predicted the best of the 5 models. For which model is the out-of-sample classification error smallest?

In [14]:
run_experiment(D_train, D_test)

3 (0.44, 0.41999999999999998)
4 (0.32000000000000001, 0.41599999999999998)
5 (0.080000000000000002, 0.188)
6 (0.040000000000000001, 0.084000000000000005)
7 (0.040000000000000001, 0.071999999999999995)


Answer: [**e**] $k=7$

### Problem 3

Reverse the role of training and validation sets; now training with the last 10 examples and validating with the first 25 examples. For which model is the classification error on the validation set smallest?

In [33]:
run_experiment(D_val, D_train)

3 (0.40000000000000002, 0.28000000000000003)
4 (0.29999999999999999, 0.35999999999999999)
5 (0.20000000000000001, 0.20000000000000001)
6 (0.0, 0.080000000000000002)
7 (0.0, 0.12)


Answer: [**d**] $k=6$

### Problem 4

Once again, evaluate the out-of-sample classification error using `out.dta` on the 5 models to see how well the validation set predicted the best of the 5 models. For which model is the out-of-sample classifcation error smallest?

In [34]:
run_experiment(D_val, D_test)

3 (0.40000000000000002, 0.39600000000000002)
4 (0.29999999999999999, 0.38800000000000001)
5 (0.20000000000000001, 0.28399999999999997)
6 (0.0, 0.192)
7 (0.0, 0.19600000000000001)


Answer: [**d**] $k=6$

### Problem 5

What values are closest in Euclidean distance to the out-of-sample classification error obtained for the model chosen in Problems 1 and 3, respectively?

`0.072`, `0.192`

Answer: [**b**] 0.1, 0.2

## Validation Bias

### Problem 6

Let $e_1$ and $e_2$ be independent random variables, distributed uniformly over the interval $[0,1]$. Let $e = \min(e_1, e_2)$. The expected values of $e_1, e_2, e$ are closest to:

In [49]:
# Generate 100000 pairs of e_1 and e_2
e = np.random.rand(100000,2)
# Take minimum of e_1 and e_2
e_min = np.min(e, axis=1)
e = np.column_stack((e, e_min))
# Compute mean
print(np.mean(e, axis=0))

[ 0.49905187  0.50138884  0.33330751]


Answer: [**d**] 0.5, 0.5, 0.4

## Cross Validation

### Problem 7

You are given the data points $(x, y): (-1, 0), (\rho, 1), (1, 0), \rho \geq 0$, and a choice between two models: constant $\{h_0(x) = b\}$ and linear $\{h_1(x) = ax + b\}$. For which value of $\rho$ would the two models be tied using leave-one-out cross-validation with the squared error measure?

Total error for $h_0 = \frac{3}{2}$

Total error for $h_1 = \left(\frac{2}{(\rho+1)}\right)^2 + \left(\frac{-2}{(\rho-1)}\right)^2 + 1$

Set the two equal to each other and solve for $\rho$:

$\left(\frac{2}{(\rho+1)}\right)^2 + \left(\frac{-2}{(\rho-1)}\right)^2 + 1 = \frac{3}{2}$

Go ahead and use wolfram alpha to solve this analytically...

Answer: [**c**] $\sqrt{9+4\sqrt{6}}$

## PLA vs. SVM

In the following problems, we compare PLA to SVM with hard margin on linearly separable data sets. For each run, you will create your own target function $f$ and data set $\mathcal{D}$. Take $d = 2$ and choose a random line in the plane as your target function $f$ (do this by takking two random, uniformly distributed points on $[-1,1] \times [-1, 1]$ and taking the line passing through them), where one side of the line maps to +1 and the other maps to -1. Choose the inputs $\mathbf{x}_n$ of the data set as random points in $\mathcal{X} = [-1,1] \times [-1, 1]$, and evaluate the target function on each $\mathbf{x}_n$ to get the corresponding output $y_n$. If all data points are on the side of the line, discard the run and start a new run.

Start PLA with the all-zero vector and pick the misclassified point for each PLA iteration at random. Run PLA to find the final hypothesis $g_\text{PLA}$ and measure the disagreement between $f$ and $g_\text{PLA}$ as $\mathbb{P}[f(\mathbf{x}) \neq g_\text{PLA}(\mathbf{x})]$ (you can either calculate this exactly, or approximate it by generating a sufficiently large, separate set of points to evaluate it). Now, run SVM on the same data to find the final hypothesis $g_\text{SVM}$ by solving

$\min_{\mathbf{w},b} \qquad \frac{1}{2}\mathbf{w}^T\mathbf{w}$

$\text{s.t.} \qquad y_n(\mathbf{w}^T\mathbf{x}_n + b) \geq 1$

using quadratic programming on the primal or the dual problem, or using an SVM package. Measure the disagreement between $f$ and $g_\text{SVM}$ as $\mathbb{P}[f(\mathbf{x}) \neq g_\text{SVM}(\mathbf{x})]$, and count the number of support vectors you get in each run.



In [62]:
from sklearn.svm import SVC

def generate_line():
    x_rand = np.random.uniform(-1, 1, size=(2,2))
    slope = (x_rand[0,1] - x_rand[0,0])/(x_rand[1,1] - x_rand[1,0])
    intercept = x_rand[0,1] - slope*x_rand[1,1]
    W_true = np.array([[intercept], [slope], [-1]])
    return W_true

def generate_points(W_true, N=100):
    ones = np.ones((1, N))
    X = np.vstack((ones, np.random.uniform(-1,1, size=(2,N))))
    Y = np.sign(np.dot(W_true.T, X)).T
    if np.all(Y == -1) or np.all(Y == 1):
        return generate_points(W_true, N)
    return X, Y

def run_pla(X, Y):
    W = np.zeros((3,1))
    Y_hat = np.sign(np.dot(W.T,X)).T
    missed = np.where(Y_hat != Y)[0]
    while missed.shape[0] > 0:
        sel = np.random.choice(missed)
        X_sel = X.T[[sel],:]
        Y_sel = Y[[sel],:]
        W = W + Y_sel*X_sel.T
        Y_hat = np.sign(np.dot(W.T,X)).T
        missed = np.where(Y_hat != Y)[0]
    return W

def estimate_e_out(W_true, W_pred, svm):
    X, Y = generate_points(W_true, N=10000)
    Y_hat = np.sign(np.dot(W_pred.T, X)).T
    pla_error = np.sum(Y_hat != Y)/10000
    svm_preds = svm.predict(X.T).reshape(10000,1)
    svm_error = np.sum(svm_preds != Y)/10000
    return pla_error, svm_error

def run_experiment(points = 10, trials = 1000):
    counts = np.zeros(1000)
    support = np.zeros(1000)
    for i in range(1000):
        W_true = generate_line()
        X, Y = generate_points(W_true, N=points)
        W_pred = run_pla(X, Y)
        svm = SVC(kernel='linear', C=np.inf)
        svm.fit(X.T, Y.ravel())
        pla_error, svm_error = estimate_e_out(W_true, W_pred, svm)
        counts[i] = svm_error < pla_error
        support[i] = np.sum(svm.n_support_)
    return np.sum(counts)/1000, np.sum(support)/1000

print("N = 10", run_experiment())
print("N = 100", run_experiment(points=100))

N = 10 (0.621, 2.8359999999999999)
N = 100 (0.625, 2.9980000000000002)


### Problem 8

Answer: [**c**] 60%

### Problem 9

Answer: [**d**] 65%

### Problem 10

Answer: [**b**] 3